In [ ]:
import pandas as pd
from plotly import express as px
pd.options.plotting.backend = 'plotly'

In [ ]:
from plotly import graph_objects as go

In [ ]:
ifrs = {
    'ifr_0': 0.0,
    'ifr_5': 0.0,
    'ifr_10': 0.0,
    'ifr_15': 2.6e-5,
    'ifr_20': 2.6e-5,
    'ifr_25': 2.6e-5,
    'ifr_30': 2.6e-5,
    'ifr_35': 5.8e-5,
    'ifr_40': 5.8e-5,
    'ifr_45': 5.8e-5,
    'ifr_50': 14.6e-5,
    'ifr_55': 14.6e-5,
    'ifr_60': 24.6e-5,
    'ifr_65': 24.6e-5,
    'ifr_70': 24.6e-5,
    'ifr_75': 5e-3,
}
ifr_series = pd.Series({int(k[4:]): v for k, v in ifrs.items()})

In [ ]:
odriscoll = pd.Series(
    {
        0: 0.003,
        5: 0.001,
        10: 0.001,
        15: 0.003,
        20: 0.006,
        25: 0.013,
        30: 0.024,
        35: 0.04,
        40: 0.075,
        45: 0.121,
        50: 0.207,
        55: 0.323,
        60: 0.456,
        65: 1.075,
        70: 1.674,
        75: 3.203,
        80: 8.292,
    }
) / 100.0

In [ ]:
erikstrup = pd.Series(
    {
        (17 + 36) / 2: 2.6,
        (35 + 51) / 2: 5.8,
        (51 + 61) / 2: 14.6,
        (61 + 73) / 2: 24.6,
    }
) / 1e5

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=odriscoll.index + 2.5, y=odriscoll))
fig.add_trace(go.Scatter(x=erikstrup.index + 2.5, y=erikstrup))
fig.update_yaxes(type='log')